# Class Workbook

## In class activity

In [162]:
import numpy as np
import pandas as pd
import math
import statsmodels.api as sm1
from plotnine import *
import statsmodels.formula.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
                        summarize ,
                        poly)
from sklearn.base import clone
from functools import partial
from ISLP.models import sklearn_sm
from sklearn.model_selection import \
     (cross_validate,
      KFold,
      ShuffleSplit)

### Ames House Data

In [163]:
ames_raw=pd.read_csv("ames_raw.csv")

ames_raw['SalePricelog'] = np.log(ames_raw['SalePrice'])

Let's revisit the AmesHousing data.  As we did in the previous class, I will split the data into before 2008 and after.  The data up to 2008 will be the training data and after 2008 will be the testing data.

In [164]:
ames_raw_2009, ames_raw_2008= ames_raw.query('`Yr Sold`>=2008').copy(), ames_raw.query('`Yr Sold` <2008').copy()

* For the regression model fit, use bootstrap to compare the standard error estimates to the model based estimates.

Your code:

In [165]:
ames_raw_2008

pd.set_option('display.max_rows', None)
print(ames_raw_2008.isnull().sum())

# The follwing variables have lots of NA
# Lot Frontage        213
# Alley              1235
# Fireplace Qu        622
# Pool QC            1309
# Fence              1070
# Misc Feature       1280

pd.reset_option('display.max_rows')

Order                 0
PID                   0
MS SubClass           0
MS Zoning             0
Lot Frontage        213
Lot Area              0
Street                0
Alley              1235
Lot Shape             0
Land Contour          0
Utilities             0
Lot Config            0
Land Slope            0
Neighborhood          0
Condition 1           0
Condition 2           0
Bldg Type             0
House Style           0
Overall Qual          0
Overall Cond          0
Year Built            0
Year Remod/Add        0
Roof Style            0
Roof Matl             0
Exterior 1st          0
Exterior 2nd          0
Mas Vnr Type         12
Mas Vnr Area         12
Exter Qual            0
Exter Cond            0
Foundation            0
Bsmt Qual            33
Bsmt Cond            33
Bsmt Exposure        35
BsmtFin Type 1       33
BsmtFin SF 1          0
BsmtFin Type 2       33
BsmtFin SF 2          0
Bsmt Unf SF           0
Total Bsmt SF         0
Heating               0
Heating QC      

In [166]:
pd.reset_option('display.max_rows')

In [167]:
# remove variables with lots of missing variables
columns_to_exclude = ['Lot Frontage', 'Alley', 'Fireplace Qu', 'Pool QC', 'Fence', 'Misc Feature']
ames_raw_2008_sub = ames_raw_2008.drop(columns=columns_to_exclude)

# remove missing values from the sub data set
ames_raw_2008_sub.dropna(inplace=True)
np.shape(ames_raw_2008_sub)

# check typs of each variable
ames_raw_2008_sub.dtypes 

# transform object class to category to create dummy variable using MS class
object_columns = ames_raw_2008_sub.select_dtypes(include='object').columns
ames_raw_2008_sub[object_columns] = ames_raw_2008_sub[object_columns].astype('category')

# reset Index prepating for the Bootstrap
ames_raw_2008_sub.reset_index(drop=True, inplace=True)

In [184]:
# chose predictors from sacatter plots
formula = "np.log(SalePrice) ~ 1 + Q('Year Built') + Q('Year Remod/Add') + Q('Total Bsmt SF') + Q('1st Flr SF') + Q('2nd Flr SF') + Q('Gr Liv Area') + Q('Full Bath') + Q('TotRms AbvGrd') + Q('Garage Area')"

# Fit the linear regression model
lmfit_2008 = sm.ols(formula, ames_raw_2008_sub).fit()
print(lmfit_2008.summary())

                            OLS Regression Results                            
Dep. Variable:      np.log(SalePrice)   R-squared:                       0.771
Model:                            OLS   Adj. R-squared:                  0.769
Method:                 Least Squares   F-statistic:                     449.8
Date:                Fri, 09 Feb 2024   Prob (F-statistic):               0.00
Time:                        14:20:30   Log-Likelihood:                 334.41
No. Observations:                1213   AIC:                            -648.8
Df Residuals:                    1203   BIC:                            -597.8
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              -1.4076    

In [188]:
def bootstrap_se(df, formula, n_bootstraps):
    bootstraps_se = []
    np.random.seed(123)

    for _ in range(n_bootstraps):
        # Sample with replacement
        sample = df.sample(n=len(df), replace=True)
        
        # Fit the model on the bootstrap sample
        model = sm.ols(formula, data=sample).fit()
        
        # Store the standard errors of the coefficients
        bootstraps_se.append(model.bse)
    
    return pd.DataFrame(bootstraps_se)

In [189]:
# Perform bootstrapping
myformula= "np.log(SalePrice) ~ 1 + Q('Year Built') + Q('Year Remod/Add') + Q('Total Bsmt SF') + Q('1st Flr SF') + Q('2nd Flr SF') + Q('Gr Liv Area') + Q('Full Bath') + Q('TotRms AbvGrd') + Q('Garage Area')"
bootstraps_se = bootstrap_se(ames_raw_2009, myformula, 1000)

# Calculate the standard errors from the bootstrap samples
bootstrap_standard_errors = bootstraps_se.std()

print("Bootstrap Standard Errors:")
print(bootstrap_standard_errors)

Bootstrap Standard Errors:
Intercept              0.048825
Q('Year Built')        0.000021
Q('Year Remod/Add')    0.000026
Q('Total Bsmt SF')     0.000002
Q('1st Flr SF')        0.000023
Q('2nd Flr SF')        0.000023
Q('Gr Liv Area')       0.000023
Q('Full Bath')         0.001073
Q('TotRms AbvGrd')     0.000454
Q('Garage Area')       0.000003
dtype: float64


Your answer:

~~~
Please write your answer in full sentences.

SE from the Bootstrap method is smaller than model based SE, which suggests that SE from Bootstrap has more confidence in the coefficient estimate than model based estimate.
~~~


* Use cross validation  to decide which model has a good predictive accuracy.  Does the result hold true for the prediction of future data?

Your code:

In [190]:
from sklearn.model_selection import KFold
import statsmodels.formula.api as sm
from sklearn.metrics import mean_squared_error

In [212]:
kf = KFold(n_splits=10, shuffle=True, random_state=123)

mse_model1 = []
mse_model2 = []

for train_index, validation_index in kf.split(ames_raw_2008):
    train_data, validation_data = ames_raw_2008.iloc[train_index], ames_raw_2008.iloc[validation_index]
    
    # Model 1
    formula1 = "np.log(SalePrice) ~ 1 + Q('Year Built') + Q('Year Remod/Add') + Q('Total Bsmt SF') + Q('1st Flr SF') + Q('2nd Flr SF') + Q('Gr Liv Area') + Q('Full Bath') + Q('TotRms AbvGrd') + Q('Garage Area')"
    model1 = sm.ols(formula1, data=train_data).fit()
    predictions1 = model1.predict(validation_data)
    predictions1= predictions1.fillna(predictions1.mean())
    mse_model1.append(mean_squared_error(np.log(validation_data['SalePrice']), predictions1))
    
    # Model 2
    formula2 = "np.log(SalePrice) ~1 + Q('Total Bsmt SF') + Q('1st Flr SF') + Q('2nd Flr SF') + Q('Gr Liv Area') + Q('Full Bath') + Q('TotRms AbvGrd') + Q('Garage Area')"
    model2 = sm.ols(formula2, data=train_data).fit()
    predictions2 = model2.predict(validation_data)
    predictions2 = predictions2.fillna(predictions2.mean())
    mse_model2.append(mean_squared_error(np.log(validation_data['SalePrice']), predictions2))



# Average MSE
avg_mse_model1 = np.mean(mse_model1)
avg_mse_model2 = np.mean(mse_model2)

print(f"Model 1 Average MSE: {avg_mse_model1}")
print(f"Model 2 Average MSE: {avg_mse_model2}")

Model 1 Average MSE: 0.036273097922835534
Model 2 Average MSE: 0.04817430226527726


Your answer:

~~~
Please write your answer in full sentences.

From the 10-fold cross validation, MSE got smaller after adding variables regarging year on the model.   
I created validation data sets through CV methods to expect that we can predict the MSE for the test data set (future data sets), but I think this is not always true.
~~~


* Using `knn.reg` fit KNN regression model.  Use cross validation to decide which K to use.

Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~


* Which model performs better on your training data?  Which model performs better in your future prediction?

Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~

## Problem Set

### Bootstrap

We will now investigate numerically the probability that a bootstrap sample of size n = 100 contains the jth observation. Here j = 4. We first create an array store with values that will subsequently
be overwritten using the function np.empty(). We then repeatedly create bootstrap samples, and each time we record whether or not the fifth observation is contained in the bootstrap
sample.

We will investigate numerically the probability that a bootstrap sample of size n = 100 contains the jth observation. Here j = 4. We repeatedly create bootstrap samples, and each time
we record whether or not the fourth observation is contained in the bootstrap sample.

Here is the code to get you going

In [ ]:
rng = np.random.default_rng (10)
store = np.empty (10000)
for i in range (10000):
    store[i] = np.sum(rng.choice(100, 100, replace=True) == 4) > 0


np.mean(store)

0.6362

Comment on the results obtained.


Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~

### CV

We will now perform cross-validation on a simulated data set.
(a) Generate a simulated data set as follows:

In [ ]:
rng = np.random.default_rng(1)
x = rng.normal(size=100)
y = x - 2 * x**2 + rng.normal(size=100)

In this data set, what is n and what is p? Write out the model
used to generate the data in equation form.


Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~


(b) Create a scatterplot of X against Y . Comment on what you find.


Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~

(c) Set a random seed, and then compute the LOOCV errors that result from fitting the following four models using least squares:

i.   $Y = \beta_0 + \beta_1X + \epsilon$
ii.  $Y = \beta_0 + \beta_1X + \beta_2X^2 + \epsilon$
iii. $Y = \beta_0 + \beta_1X + \beta_2X^2 + \beta_3X^3 + \epsilon$
iv.  $Y = \beta_0 + \beta_1X + \beta_2X^2 + \beta_3X^3 + \beta_4X^4 + \epsilon$.

Note you may find it helpful to use the `data.frame()` function
to create a single data set containing both $X$ and $Y$ .


Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~


(d) Repeat (c) using another random seed, and report your results.
Are your results the same as what you got in (c)? Why?

Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~

(e) Which of the models in (c) had the smallest LOOCV error? Is this what you expected? Explain your answer.

Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~


(f) Comment on the statistical significance of the coefficient estimates that results from fitting each of the models in (c) using least squares. Do these results agree with the conclusions drawn based on the cross-validation results?


Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~

### Default 1

In Chapter 4, we used logistic regression to predict the probability of default using income and balance on the `Default` data set. We will now estimate the test error of this logistic regression model using the validation set approach. Do not forget to set a random seed before beginning your analysis.

(a) Fit a logistic regression model that uses income and balance to predict default.

Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~


(b) Using the validation set approach, estimate the test error of this model. In order to do this, you must perform the following steps:

i. Split the sample set into a training set and a validation set.

Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~

ii. Fit a multiple logistic regression model using only the training observations.

Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~


iii. Obtain a prediction of default status for each individual in the validation set by computing the posterior probability of default for that individual, and classifying the individual to the default category if the posterior probability is greater than 0.5.


Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~


iv. Compute the validation set error, which is the fraction of the observations in the validation set that are misclassified.

Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~


(c) Repeat the process in (b) three times, using three different splits of the observations into a training set and a validation set. Comment on the results obtained.


Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~


(d) Now consider a logistic regression model that predicts the probability of default using income, balance, and a dummy variable for student. Estimate the test error for this model using the validation set approach. Comment on whether or not including a dummy variable for student leads to a reduction in the test error rate.


Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~

### Default 2

We continue to consider the use of a logistic regression model to
predict the probability of default using income and balance on the
Default data set. In particular, we will now compute estimates for the
standard errors of the income and balance logistic regression coefficients
in two different ways: (1) using the bootstrap, and (2) using the
standard formula for computing the standard errors in the sm.GLM()
function. Do not forget to set a random seed before beginning your
analysis.

(a) Using the `summarize()` and `sm.GLM()` functions, determine the
estimated standard errors for the coefficients associated with
income and balance in a multiple logistic regression model that
uses both predictors.
Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~


(b) Write a function, `boot_fn()`, that takes as input the Default data
set as well as an index of the observations, and that outputs
the coefficient estimates for income and balance in the multiple
logistic regression model.

Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~

(c) Following the bootstrap example in the lab, use your `boot_fn()`
function to estimate the standard errors of the logistic regression
coefficients for income and balance.


Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~


(d) Comment on the estimated standard errors obtained using the `sm.GLM()` function and using the bootstrap.

Your answer:

~~~
Please write your answer in full sentences.


~~~

### Boston housing data set

We will now consider the Boston housing data set, from the ISLR library.

In [ ]:
Boston = load_data("Boston")

(a) Based on this data set, provide an estimate for the population mean of medv. Call this estimate $\hat{\mu}$.

Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~


(b) Provide an estimate of the standard error of $\hat{\mu}$. Interpret this result.
Hint: We can compute the standard error of the sample mean by dividing the sample standard deviation by the square root of the number of observations.

Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~


(c) Now estimate the standard error of $\hat{\mu}$ using the bootstrap. How does this compare to your answer from (b)?

Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~

(d) Based on your bootstrap estimate from (c), provide a 95 % confidence
interval for the mean of medv. Compare it to the results
obtained by using `Boston['medv'].std()` and the two standard
error rule (3.9).
Hint: You can approximate a 95% confidence interval using the formula $[\hat{\mu} − 2SE(\hat{\mu}), \hat{\mu} + 2SE(\hat{\mu})]$.

Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~


(e) Based on this data set, provide an estimate, $\hat{\mu}_{med}$, for the median value of medv in the population.

Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~


(f) We now would like to estimate the standard error of $\hat{\mu}_{med}$. Unfortunately, there is no simple formula for computing the standard error of the median. Instead, estimate the standard error of the median using the bootstrap. Comment on your findings.

Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~



(g) Based on this data set, provide an estimate for the tenth percentile
of medv in Boston census tracts. Call this quantity $\hat{\mu}_{0.1}$.
(You can use the `np.percentile()` function.)

Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~


(h) Use the bootstrap to estimate the standard error of $\hat{\mu}_{0.1}$. Comment on your findings.

Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~

## Additional Material

### Cross Validation for Predictive Modeling Platforms in Python

#### scikit-learn

Using platforms like scikit-learn is helpful for things such as cross validation.
There are variants of CV implemented to be used off the shelf
https://scikit-learn.org/stable/modules/cross_validation.html

There are many metrics you can choose from
https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

In [ ]:
ames_raw_comp=ames_raw.loc[:,["SalePrice","Gr Liv Area","Lot Area","Total Bsmt SF"]].dropna()
X=ames_raw_comp.loc[:,["Gr Liv Area","Lot Area","Total Bsmt SF"]]
y=ames_raw_comp.loc[:,"SalePrice"]
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn import svm
from sklearn.linear_model import LinearRegression
clf =  LinearRegression()
## One metric
scores = cross_val_score(clf, X, y, cv=5,scoring="r2")
## Multiple metrics
scoring = ['r2', 'max_error']
scores = cross_validate(clf, X, y, scoring=scoring)

#### PySpark

[Apache Spark](https://spark.apache.org/docs/3.1.3/api/python/index.html) is a popular large data handling platform.  Over the years, they built Machine Learning capabilities in MLlib.

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.linalg import Vectors
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, CrossValidatorModel
import tempfile

In [ ]:
spark = SparkSession.builder.appName("CV_test").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/09 11:10:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
dataset = spark.createDataFrame(
    [(Vectors.dense([0.0]), 0.0),
     (Vectors.dense([0.4]), 1.0),
     (Vectors.dense([0.5]), 0.0),
     (Vectors.dense([0.6]), 1.0),
     (Vectors.dense([1.0]), 1.0)] * 10,
    ["features", "label"])
lr = LogisticRegression()
grid = ParamGridBuilder().addGrid(lr.maxIter, [0, 1]).build()
evaluator = BinaryClassificationEvaluator()
cv = CrossValidator(estimator=lr, estimatorParamMaps=grid, evaluator=evaluator,
    parallelism=2)
cvModel = cv.fit(dataset)
cvModel.getNumFolds()
cvModel.avgMetrics[0]
evaluator.evaluate(cvModel.transform(dataset))

24/02/09 11:11:03 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
Java HotSpot(TM) 64-Bit Server VM warning: CodeCache is full. Compiler has been disabled.
Java HotSpot(TM) 64-Bit Server VM warning: Try increasing the code cache size using -XX:ReservedCodeCacheSize=


CodeCache: size=131072Kb used=35459Kb max_used=35459Kb free=95612Kb
 bounds [0x00000001081e0000, 0x000000010a4c0000, 0x00000001101e0000]
 total_blobs=13933 nmethods=12816 adapters=1027
 compilation: disabled (not enough contiguous free space left)


0.8333333333333333

In [ ]:
spark.stop()

#### Resampling  using h2o

On H2O cross validation is embedded in the function calling so you don't need to worry about learning a new wrapper on top of your estimator.
https://docs.h2o.ai/h2o/latest-stable/h2o-docs/cross-validation.html

In [ ]:
#pip install requests
#pip install tabulate
#pip uninstall h2o
#pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o
# load packages and data
import h2o
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

##### Starting H2O

To use H2O you need to instantiate it.

In [ ]:
# nthreads specifies number of threads. -1 means use all the CPU cores.
# max_mem_size specifies the maximum amount of RAM to use.
localH2O= h2o.init(nthreads = -1, max_mem_size="4g")

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_401"; Java(TM) SE Runtime Environment (build 1.8.0_401-b10); Java HotSpot(TM) 64-Bit Server VM (build 25.401-b10, mixed mode)
  Starting server from /Users/yutatsukumo/anaconda3/lib/python3.11/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/zx/tjnnhhbx01x_b6njsk8c9p4c0000gp/T/tmp42falxjc
  JVM stdout: /var/folders/zx/tjnnhhbx01x_b6njsk8c9p4c0000gp/T/tmp42falxjc/h2o_yutatsukumo_started_from_python.out
  JVM stderr: /var/folders/zx/tjnnhhbx01x_b6njsk8c9p4c0000gp/T/tmp42falxjc/h2o_yutatsukumo_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.3
H2O_cluster_version_age:,1 month and 19 days
H2O_cluster_name:,H2O_from_python_yutatsukumo_sb1u9t
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.541 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


##### CV with  H2O

In [ ]:
# Import the prostate dataset
prostate = h2o.import_file("http://s3.amazonaws.com/h2o-public-test-data/smalldata/prostate/prostate.csv.zip")

# Set the predictor names and the response column name
response = "CAPSULE"
predictors = prostate.names[3:8]

# Convert the response column to a factor
prostate['CAPSULE'] = prostate['CAPSULE'].asfactor()

# Train a GBM model setting nfolds to 5
prostate_gbm = H2OGeneralizedLinearEstimator(nfolds = 5, seed = 1)
prostate_gbm.train(x=predictors, y=response, training_frame=prostate)

# AUC of cross-validated holdout predictions
prostate_gbm.auc(xval=True)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
glm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


0.7790697716207187

##### Shut down H2O

In [ ]:
h2o.cluster().shutdown(prompt =False)

H2O session _sid_8ba4 closed.
